In [37]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [38]:
#start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A_all=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())
tmp_flag = (workload_A_all != 0).any(axis=0)
workload_A = workload_A_all.loc[:, tmp_flag]
#workload_A
workload_A = workload_A.loc[:, ~(workload_A == workload_A.iloc[0]).all()]

workload_A = workload_A.loc[:, workload_A.isin([0]).mean() < .6]

knobs_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']


#print(workload_A_all)
#print("--- %s seconds ---" % (time.time() - start_time))

In [39]:
# Normalization
from sklearn.preprocessing import MinMaxScaler

tmpframe = workload_A.T.copy()

tmpframe1 = tmpframe.tail(314-1).T
tmpframe2 = tmpframe.head(1).T

knobs_metrics_name = tmpframe1.columns
work_id_name = tmpframe2.columns

tmp_workload_knobs_metrics = (workload_A[knobs_metrics_name]).values

dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

workload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_knobs_metrics)
tmp_workload_id = (workload_A[work_id_name]).values

workload_knobs_metrics_Norm_id = np.hstack((tmp_workload_id,workload_knobs_metrics_Norm))

col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_A = pd.DataFrame(data=workload_knobs_metrics_Norm_id,columns=col_list)

latencyArray = (workload_A[['latency']]).values

latencyScalar = MinMaxScaler().fit(latencyArray)


In [40]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = norm_workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = norm_workload_A[norm_workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))
#workload_A_list[1]

--- 0.18223047256469727 seconds ---


In [41]:
start_time = time.time()

#Step3 Get metrics

tmpframe = norm_workload_A.T.copy()
#tmpframe = tmpframe.tail(585-1-12).T
#tmpframe = tmpframe.tail(414-1-12).T
#tmpframe = tmpframe.tail(372-1-12).T
tmpframe = tmpframe.tail(314-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))
(tmpframe)

--- 1.6101861000061035 seconds ---


,latency,mimic_cpu_util,driver.BlockManager.memory.maxMem_MB.avg,driver.BlockManager.memory.maxOnHeapMem_MB.avg,driver.BlockManager.memory.memUsed_MB.avg,driver.BlockManager.memory.onHeapMemUsed_MB.avg,driver.BlockManager.memory.remainingMem_MB.avg,driver.BlockManager.memory.remainingOnHeapMem_MB.avg,driver.DAGScheduler.job.activeJobs.avg,driver.DAGScheduler.job.allJobs.avg_increase,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,0.0134949,0.0246714,0.016629,0.016629,0.000916009,0.000916009,0.0236648,0.0236648,0.677154,0.118812,...,0.0477606,0.048065,0.012964,0.0357318,0.000638225,0.0480704,0.0232684,0.0395533,0.0929657,0.065145
1,0.0009664,0.389111,0.783374,0.783374,0.00477507,0.00477507,0.78413,0.78413,0.221831,0.0594059,...,0.335875,0.292873,0.0780634,0.240153,0.00950681,0.313699,0.383333,0.351963,0.439727,0.207286
2,0.00168567,0.340512,0.387842,0.387842,0.0025399,0.0025399,0.391883,0.391883,0.433099,0.148515,...,0.184134,0.14993,0.0513811,0.124257,0.00940101,0.186962,0.105556,0.218565,0.376413,0.180328
3,0.00230275,0.262531,0.308037,0.308037,0.00252109,0.00252109,0.31266,0.31266,0.363682,0.178218,...,0.15389,0.116555,0.0415938,0.101355,0,0.172973,0.0972222,0.169665,0.310851,0.139158
4,0.00617735,0.0831844,0.072887,0.072887,0.00128761,0.00128761,0.0794478,0.0794478,0.586268,0.207921,...,0.140285,0.0765939,0.0193498,0.058595,0,0.12488,0.1875,0.0738212,0.171111,0.0924122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18344,0.056588,0.53047,0.748011,0.748011,0.0174607,0.0174607,0.746732,0.746732,0.782774,0.19802,...,0.297283,0.314484,0.490009,0.240323,0,0.623782,0.192389,0.475659,0.0407299,0.0571209
18345,0.0374673,0.791403,0.88653,0.88653,0.0152282,0.0152282,0.884647,0.884647,0.768306,0.207921,...,0.102397,0.165727,0.603045,0.123939,0,0.198524,0.0703551,0.715209,0.0550059,0.067304
18346,0.0631478,0.446528,0.410449,0.410449,0.00664394,0.00664394,0.413586,0.413586,0.794761,0.217822,...,0.273018,0.279591,0.431849,0.228121,0,0.600574,0.179327,0.335596,0.0341861,0.0477664
18347,0.080886,0.345435,0.313089,0.313089,0.00365573,0.00365573,0.317468,0.317468,0.800002,0.039604,...,0.255025,0.233936,0.37855,0.18332,0.00910968,0.54745,0.152694,0.261588,0.0265045,0.0464286


In [42]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

#knobs_metrics_name = tmpframe1.columns
#work_id_name = tmpframe2.columns

tmp_workload_B_knobs_metrics = (workload_B[knobs_metrics_name]).values

#dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

Bworkload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_B_knobs_metrics)
Btmp_workload_id = (workload_B[work_id_name]).values

Bworkload_knobs_metrics_Norm_id = np.hstack((Btmp_workload_id,Bworkload_knobs_metrics_Norm))

#col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_B = pd.DataFrame(data=Bworkload_knobs_metrics_Norm_id,columns=col_list)



print("--- %s seconds ---" % (time.time() - start_time))
norm_workload_B

--- 0.32318568229675293 seconds ---


,workload id,k1,k2,k3,k4,k5,k6,k7,k8,s1,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.010101,...,0.187263,0.434244,0.0489238,0.284526,1.30494,0.162126,1.18333,0.223063,0.377736,0.225861
1,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.157029,0.369936,0.0443326,0.22023,1.0009,0.172732,0.857143,0.19635,0.317073,0.182314
2,1-1,0.134615,0.0294118,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.0767531,0.245833,0.0152169,0.147634,0.861225,0.0733324,0.339815,0.0693224,0.146857,0.13885
3,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.167855,0.339363,0.0470671,0.228134,0.958487,0.180253,1.68056,0.256706,0.340859,0.198389
4,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.158224,0.109943,0.0385281,0.0939092,0.0074373,0.202941,0.0873016,0.201799,0.323645,0.148199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,14-1,0.653846,1,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00161311,0.0686614,0.203777,0.0459204,7.07352e-06,0.0267308,0,0.287024,0.759105,0.0854303
596,14-1,0.192308,0.294118,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00112479,0.021567,0.150025,0.159283,0,0.0119027,0.0208333,0.210566,0.537258,0.0477424
597,14-1,0.269231,0.411765,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00252511,0.0370058,0.141262,0.314006,1.83154e-05,0.0180185,0.0257937,0.259081,0.63011,0.0625165
598,14-1,0.653846,1,0,0,0.0769231,0.919048,0,0.4,0.0909091,...,0.00267613,0.0738297,0.196381,0.514412,1.4147e-05,0.0316406,0.0472222,0.3592,0.795794,0.0872622


In [43]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = norm_workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = norm_workload_B[norm_workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.05448269844055176 seconds ---


In [44]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

workload_B_mapping_data_list = []
workload_B_target_list = []

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

workload_B_mapping_data2 = []
workload_B_target2 = []

workload_B_mapping_data3 = []
workload_B_target3 = []

workload_B_mapping_data4 = []
workload_B_target4 = []

workload_B_mapping_data5 = []
workload_B_target5 = []

workload_B_mapping_data6 = []
workload_B_target6 = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])

workload_B_mapping_data_list.append(workload_B_mapping_data)   
workload_B_target_list.append(workload_B_target)

for i in range(num_workloads_B):
    workload_B_mapping_data2.append(pd.concat([(workload_B_list[i][0:4]),(workload_B_list[i][5:6])]))
    workload_B_target2.append((workload_B_list[i][4:5]))

workload_B_mapping_data_list.append(workload_B_mapping_data2)   
workload_B_target_list.append(workload_B_target2)    
    
for i in range(num_workloads_B):
    workload_B_mapping_data3.append(pd.concat([(workload_B_list[i][0:3]),(workload_B_list[i][4:6])]))
    workload_B_target3.append((workload_B_list[i][3:4]))
    
workload_B_mapping_data_list.append(workload_B_mapping_data3)   
workload_B_target_list.append(workload_B_target3)

for i in range(num_workloads_B):
    workload_B_mapping_data4.append(pd.concat([(workload_B_list[i][0:2]),(workload_B_list[i][3:6])]))
    workload_B_target4.append((workload_B_list[i][2:3]))

workload_B_mapping_data_list.append(workload_B_mapping_data4)   
workload_B_target_list.append(workload_B_target4)

for i in range(num_workloads_B):
    workload_B_mapping_data5.append(pd.concat([(workload_B_list[i][0:1]),(workload_B_list[i][2:6])]))
    workload_B_target5.append((workload_B_list[i][1:2]))

workload_B_mapping_data_list.append(workload_B_mapping_data5)   
workload_B_target_list.append(workload_B_target5)

for i in range(num_workloads_B):
    workload_B_mapping_data6.append(pd.concat([(workload_B_list[i][0:0]),(workload_B_list[i][1:6])]))
    workload_B_target6.append((workload_B_list[i][0:1]))

workload_B_mapping_data_list.append(workload_B_mapping_data6)   
workload_B_target_list.append(workload_B_target6)
    
    

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.3869333267211914 seconds ---


In [45]:
#Step4 FA for all workloads together Function
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize


start_time = time.time()

#all_metrics_data_normalized = normalize(all_metrics_data, norm='l2')
#all_metrics_data_normalized_Trans = all_metrics_data_normalized.T

#all_metrics_data_Trans = all_metrics_data.T
#all_metrics_data_normalized_Trans = normalize(all_metrics_data_Trans, norm='max')

#print(all_metrics_data_normalized_Trans.shape)
#tmp_all_transformer = FactorAnalysis(n_components=170, random_state=0)
#tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_Trans)
#print(tmp_workload_A_transformed.shape)
#print("--- %s seconds ---" % (time.time() - start_time))

def FAforAllworkloads(n_c,frame):
    all_metrics_data = frame.values

    all_metrics_data_Trans = all_metrics_data.T
    tmp_all_transformer = FactorAnalysis(n_components=n_c, random_state=0)
    tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_Trans)
    
    return tmp_workload_A_transformed
    

In [46]:
#Step5 K-means for all workloads together
from sklearn.cluster import KMeans

def KmeanForallworkloads(n_k,faworkloads):
    workload_A_list_array_Kmeans = KMeans(n_clusters=n_k, random_state=0).fit(faworkloads)
    cluster_centers = workload_A_list_array_Kmeans.cluster_centers_
    tmp_centers_index = []
    tmp_centers_metrics_name = []
    
    for i in range(n_k):
        tmp_center = cluster_centers[i]
        tmp_center_diff = tmp_center - faworkloads
        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
        tmp_center_min_idx = np.argmin(tmp_center_distance)
        tmp_centers_index.append(tmp_center_min_idx)
    
    for j in range(n_k):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
        #print(tmp_centers_index)
        #print(tmp_centers_metrics_name)

    tmp_centers_metrics_name = ['latency'] + tmp_centers_metrics_name
    print(tmp_centers_metrics_name)
    num_M = 1 + n_k
    workload_list_centers_metrics_name = []
    for n in range(58):
        workload_list_centers_metrics_name.append(tmp_centers_metrics_name)
        
    return num_M,tmp_centers_metrics_name

In [47]:
#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler



def BuildGPRForall(kernel_t,workload_A,centers_metrics_name):
    tmp_kernel = kernel_t
    
    workload_A_GPRmodels_lists = []
    workload_A_scalar_lists = []
    workload_A_normalized_m = []
    
    for i in range(58):
        tmp_GPRmodels_lists = []
        tmp_scalar_lists = []
        tmp_normalized_m = []
    
    
        tmp_Pruning_workload_A_X = ((workload_A[i])[knobs_list]).values
    
        tmp_num_centers = len(centers_metrics_name)
        
        for j in range(tmp_num_centers):
        
            tmp_Pruning_workload_A_y_m = ((workload_A[i])[centers_metrics_name[j]]).values
            tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X, tmp_Pruning_workload_A_y_m)
            tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
        workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)
    return workload_A_GPRmodels_lists
#print(workload_A_normalized_m)

In [48]:
#Step4 workload mapping Test for all workload in workload B

def FindMostSimliarIndexForTarget(workload_A,workload_mapping,GPRmodelList,center_metric_name,t_kernel,n_m):
    all_distance_list = []
    for i in range(100):
        tmp_test_workload_B_mapping = workload_mapping[i]
        tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_list]).values
    
        tmp_distance_list = []
    
        for j in range(58):
            tmp_test_workload_A = workload_A[j]
            tmp_test_workload_A_gpr_model = GPRmodelList[j]   
            tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_list]).values
        
            tmp_corr_metric_values = []
        
            for k in range(5):
                tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
                tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
                if len(tmp_check[0]) == 0:
                    tmp_gpr_predic = []
                    for v in range(nk):
                        tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[v].predict([tmp_data_point])[0])
                    tmp_corr_metric_values.append(tmp_gpr_predic)
            
                else:
                    tmp_gpr_predic = []
                    tmp_row_idx = tmp_check[0][0]
                    for n in range(n_m):
                        tmp_gpr_predic.append((((tmp_test_workload_A)[[center_metric_name[n]]]).values)[tmp_row_idx])
                    tmp_corr_metric_values.append(tmp_gpr_predic)
        
            tmp_test_workload_B_mapping_metrics_norm_value_list = []
            for m in range(nk):
                tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[center_metric_name[m]]].values

                tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_value)


            tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((n_m,5))       
            tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,n_m)).T

            up_min_list = np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
            down_min_list = np.min(tmp_corr_metric_values_arry,axis=0)
            tot_min_list = np.min(np.vstack((up_min_list,down_min_list)),axis=0)
        
            up_max_list = np.max(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
            down_max_list = np.max(tmp_corr_metric_values_arry,axis=0)
            tot_max_list = np.max(np.vstack((up_max_list,down_max_list)),axis=0)

            tmp_test_workload_B_mapping_metrics_norm_value_arry_norm = np.divide((tmp_test_workload_B_mapping_metrics_norm_value_arry - tot_min_list),(tot_max_list - tot_min_list))
            tmp_corr_metric_values_arry_norm = np.divide((tmp_corr_metric_values_arry - tot_min_list),(tot_max_list - tot_min_list))

            tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm)*(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm))
            tmp_distance_list.append(tmp_distance)
     
        all_distance_list.append(tmp_distance_list)


    
    all_min_idx_list = []
    for i in range(100):
        tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
        all_min_idx_list.append(tmp_idx)
    return all_min_idx_list

In [49]:
#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

def Concatenate(workload_A,workload_mapping,center_metric_name,t_kernel,min_idx_list):
    predictGPRmodel_list = []
    #predictScalar_list = []
    #inputScalar_list = []


    for i in range(100):
        
        #print(i)
        tmp_similar_idx = min_idx_list[i]
        tmp_similar_workload = workload_A[tmp_similar_idx]
        tmp_similar_workload_configs = ((tmp_similar_workload)[knobs_list]).values
        tmp_similar_workload_metrics = ((tmp_similar_workload)[[center_metric_name[0]]]).values
    
        tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
        tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
        tmp_workload_B_configs = workload_mapping[i][knobs_list].values
        tmp_workload_B_metrics = workload_mapping[i][[center_metric_name[0]]].values
    
        for j in range(5):
            tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
            tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_metrics[j]
            tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
            if len(tmp_check[0]) == 0:
            
                tmp_Augmented_pruned_target_configs = np.append(tmp_Augmented_pruned_target_configs,[tmp_test_workload_B_mapping_cfigs],axis=0)
                tmp_Augmented_pruned_target_metrics = np.append(tmp_Augmented_pruned_target_metrics,[tmp_test_workload_B_mapping_metrics_with_similar],axis=0)
            else:
        
                tmp_row_idx = tmp_check[0][0]
                tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
        test_workload_B_gpr_o = GaussianProcessRegressor(kernel=t_kernel).fit(tmp_Augmented_pruned_target_configs, tmp_Augmented_pruned_target_metrics)
        predictGPRmodel_list.append(test_workload_B_gpr_o)
        
    return predictGPRmodel_list

In [58]:
#Predict
def PredictMAPE(workload_target,predGPRmodelList,dscalar,center_metric_name,min_idx_list):
    rightSet = {}

    totPercent = 0
    right_count = 0
    for i in range(100):
        tmp_target_config = workload_target[i][knobs_list].values
        tmp_max = dscalar.data_max_[12]
        tmp_min = dscalar.data_min_[12]
        tmp_predict = predGPRmodelList[i].predict(tmp_target_config)
        
        #tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
        #tmp_ground_Truth = workload_B_target[i][[center_metric_name[0]]].values * (tmp_max - tmp_min) + tmp_min
        tmp_predict_re = latencyScalar.inverse_transform(tmp_predict)
        tmp_ground_Truth = latencyScalar.inverse_transform(workload_target[i][[center_metric_name[0]]].values)
        
        tmp_diff = np.abs(tmp_predict_re - tmp_ground_Truth)
        #print(tmp_predict_re)
        #print(tmp_ground_Truth)
        totPercent += tmp_diff/tmp_ground_Truth
        if (tmp_diff/tmp_ground_Truth) < 0.3:
            right_count += 1
            rightSet[i] = min_idx_list[i]
    #print('------------')

    print('MAPE: ',totPercent/100)
    print('Right Predict with absolute percentage error less than 0.3: ', right_count)
    return totPercent,rightSet

In [59]:
kernel_1 = 1.0 * RBF(length_scale=1.0) + WhiteKernel()
num_FA_reduce = [100]
num_K = [3]

result_set = {}
metric_set = {}

for i in range(len(num_FA_reduce)):
    for j in range(len(num_K)):
        a = FAforAllworkloads(num_FA_reduce[i],tmpframe)
        nk,b = KmeanForallworkloads(num_K[j],a)
        
        metric_set[(i,j)] = b
        
        d = BuildGPRForall(kernel_1,workload_A_list,b)
        
        tmp_MAPE = 0
        for n in range(6):
            e = FindMostSimliarIndexForTarget(workload_A_list,workload_B_mapping_data_list[n],d,b,kernel_1,nk)
            f = Concatenate(workload_A_list,workload_B_mapping_data_list[n],b,kernel_1,e)
            g,h = PredictMAPE(workload_B_target_list[n],f,dataScalar,b,e)
            tmp_MAPE += g[0][0]
        result_set[(i,j)] = tmp_MAPE/6
            #print(g)

['latency', 'driver.jvm.pools.Compressed-Class-Space.usage.nb_change', 'driver.jvm.PS-Scavenge.time.avg_period', 'worker_1.Processes.Blocked']
MAPE:  [[0.40467419]]
Right Predict with absolute percentage error less than 0.3:  54
MAPE:  [[0.40526294]]
Right Predict with absolute percentage error less than 0.3:  50
MAPE:  [[0.41917077]]
Right Predict with absolute percentage error less than 0.3:  50
MAPE:  [[0.52025244]]
Right Predict with absolute percentage error less than 0.3:  46
MAPE:  [[0.47134013]]
Right Predict with absolute percentage error less than 0.3:  54
MAPE:  [[0.54950181]]
Right Predict with absolute percentage error less than 0.3:  52


In [57]:
kernel_o = 1.3 * RBF(length_scale=5.0) + WhiteKernel(noise_level=0.5)
num_FA_reduce = [50]
num_K = [3]

#result_set = {}
#metric_set = {}

for i in range(len(num_FA_reduce)):
    for j in range(len(num_K)):
        a = FAforAllworkloads(num_FA_reduce[i],tmpframe)
        nk,b = KmeanForallworkloads(num_K[j],a)
        
        #metric_set[(i,j)] = b
        
        d = BuildGPRForall(kernel_1,workload_A_list,b)
        
        tmp_MAPE = 0
        for n in range(6):
            e = FindMostSimliarIndexForTarget(workload_A_list,workload_B_mapping_data_list[n],d,b,kernel_1,nk)
            f = Concatenate(workload_A_list,workload_B_mapping_data_list[n],b,kernel_1,e)
            g,h = PredictMAPE(workload_B_target_list[n],f,dataScalar,b,e)
            tmp_MAPE += g[0][0]
        #result_set[(i,j)] = tmp_MAPE/6
            #print(g)

['latency', 'worker_1.Disk_Write_KB/s.dm-0', 'worker_1.JFS_Filespace_%Used./home', 'executor.jvm.non-heap.committed.nb_change']
[[7.24276413]]
[[7.923]]
[[377.73015538]]
[[260.62625]]
[[111.90189106]]
[[109.0855]]
[[64.37914019]]
[[61.997]]
[[97.72133994]]
[[89.78475]]
[[24.94074543]]
[[24.4785]]
[[7.28493332]]
[[4.507125]]
[[126.77371789]]
[[107.993]]
[[49.24700592]]
[[42.31425]]
[[31.47504993]]
[[52.674]]
[[8.71478546]]
[[3.084]]
[[19.72321773]]
[[9.70575]]
[[17.9200017]]
[[46.812]]
[[8.20887519]]
[[24.076]]
[[10.60145982]]
[[8.29225]]
[[4.84159813]]
[[18.494]]
[[17.90642859]]
[[59.1705]]
[[58.85898827]]
[[241.722]]
[[39.73675669]]
[[39.0855]]
[[27.86079609]]
[[13.14875]]
[[20.88982357]]
[[12.77833333]]
[[45.24992846]]
[[27.9515]]
[[35.31392625]]
[[31.53075]]
[[10.53919267]]
[[17.894]]
[[21.06281888]]
[[18.4875]]
[[11.12563094]]
[[13.66325]]
[[7.4341779]]
[[7.98341667]]
[[18.73268425]]
[[28.34975]]
[[3.55468151]]
[[3.320875]]
[[883.75197727]]
[[764.95875]]
[[6.32798986]]
[[6.60758333

[[87.57310798]]
[[60.77025]]
[[10.67936368]]
[[7.5945]]
[[10.94789784]]
[[16.12075]]
[[19.9749596]]
[[75.699]]
[[6.6964888]]
[[4.20475]]
MAPE:  [[0.62131862]]
Right Predict with absolute percentage error less than 0.3:  44
[[7.23947874]]
[[8.02325]]
[[523.40867783]]
[[325.97575]]
[[200.36466919]]
[[178.701]]
[[200.60416976]]
[[339.2145]]
[[92.71693958]]
[[79.14875]]
[[31.26795005]]
[[31.522]]
[[7.29175199]]
[[6.83341667]]
[[126.50764142]]
[[109.47275]]
[[18.14973136]]
[[5.28325]]
[[50.79646933]]
[[50.653]]
[[9.27716467]]
[[2.81625]]
[[19.15718447]]
[[16.13]]
[[17.9320636]]
[[7.17325]]
[[8.27483728]]
[[2.70725]]
[[6.88939109]]
[[6.868]]
[[5.65064536]]
[[8.19625]]
[[6.42793584]]
[[54.27]]
[[337.55172328]]
[[297.344]]
[[29.40943909]]
[[32.509]]
[[36.69530354]]
[[13.72408333]]
[[20.87441656]]
[[14.725]]
[[21.20581886]]
[[12.12025]]
[[4.02968406]]
[[17.35375]]
[[10.57240853]]
[[3.12175]]
[[21.07190939]]
[[14.2635]]
[[8.57120499]]
[[8.479]]
[[7.42132795]]
[[7.93958333]]
[[47.39038945]]
[[51.

[[0.86586047]]
[[26.6985]]
[[10.67982465]]
[[13.34125]]
[[8.96494422]]
[[8.52075]]
[[19.98448561]]
[[16.9775]]
[[6.69701386]]
[[3.42175]]
MAPE:  [[0.57407974]]
Right Predict with absolute percentage error less than 0.3:  48
